In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
# autoreloads imports every time a module is called

In [3]:
from transformers import get_transformer

In [4]:
models, tokenizer = get_transformer('gpt2')

In [6]:
sent1 = "my name is [SOMEONE], and you killed my father"
sent2 = "do not go gentle into that good night"

In [9]:
import torch
from transformers import make_batch

x, lengths, _ = make_batch(tokenizer, [sent1, sent2])

In [10]:
x

tensor([[ 1820,  1438,   318, 50259,    11,   290,   345,  2923,   616,  2988],
        [ 4598,   407,   467, 10296,   656,   326,   922,  1755, 50263, 50263]])

In [11]:
from munch import Munch

args = Munch({
    'keyword_threshold': 1/3,
    'transformer_pool': 'mean', 
    'threshold_gap': 3,
})

In [33]:
from model.encoder import Encoder

ae_encoder = Encoder(args, models['encoder'], tokenizer)

In [34]:
keywords, keyword_lengths, scores, reg_loss = ae_encoder(x, lengths)

In [21]:
from model.decoder import Decoder

ae_decoder = Decoder(args, models['decoder'], tokenizer)

In [22]:
logits = ae_decoder(x, keywords, keyword_lengths, scores)

In [23]:
logits.shape

torch.Size([2, 11, 50264])

In [24]:
opt = torch.optim.Adam(ae_encoder.parameters(), lr=0.0001)

In [36]:
reg_loss.sum().backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [51]:
sentence = x
B = sentence.shape[0]
h, _ = ae_encoder.transformer(sentence)
h = ae_encoder.aggregate(h, -1)
scores = torch.sigmoid(h)
threshold = ae_encoder.get_threshold(lengths)
'''
for i in range(B):
    scores[i, lengths[i]:] = 0
'''
reg_loss = ae_encoder.get_score_loss(scores, threshold)

sorted_idx = scores.argsort(dim=-1, descending=True)
threshold_idx = sorted_idx
for i in range(B):
    threshold_idx[i, threshold[i] + ae_encoder.gap:] = 0
scores = ae_encoder.gather_with_idx(threshold_idx, scores)
keywords = sentence
keywords

In [52]:
reg_loss.sum().backward()